In [1]:
# -----------------------------------------------
# 1) Importar solo lo necesario
# -----------------------------------------------
from pathlib import Path
from tensorflow import keras

# -----------------------------------------------
# 2) Ruta del modelo (misma carpeta del notebook)
#    Si tu archivo se llama distinto, cámbialo aquí
# -----------------------------------------------
model_path = Path("modelo_resnet_entrenado.keras")

# Verificación opcional: asegurarse de que el archivo existe
assert model_path.exists(), f"No se encontró {model_path}"

# -----------------------------------------------
# 3) Cargar el modelo
# -----------------------------------------------
model = keras.models.load_model(model_path)

# -----------------------------------------------
# 4) Mostrar resumen de la arquitectura
# -----------------------------------------------
print("Arquitectura del modelo:")
model.summary()


Arquitectura del modelo:


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_4      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 7)              │         1,799 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 72,235,927 (275.56 MB)

 Trainable params: 24,060,935 (91.79 MB)

 Non-trainable params: 53,120 (207.50 KB)

 Optimizer params: 48,121,872 (183.57 MB)

In [2]:
# Código correcto:

import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

# Clases para FER2013
class_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


# Inferencia con una imagen de prueba
def preprocess_image(image_path):
    # Cargar la imagen y redimensionar al tamaño esperado
    img = keras.preprocessing.image.load_img(image_path, target_size=(224, 224))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Añadir dimensión de batch
    img_array /= 255.0  # Normalización
    return img_array

# Mostrar la imagen con la predicción
def show_image_with_prediction(image_path, predicted_class, confidence):
    img = keras.preprocessing.image.load_img(image_path)
    plt.imshow(img)
    plt.title(f"Predicción: {predicted_class} ({confidence:.4f})")
    plt.axis("off")
    plt.show()

# Ruta de la imagen de prueba
image_path = "/content/hombre_8.jpg"  # Cambia por la imagen que quieras probar
input_data = preprocess_image(image_path)

# Realizar inferencia
predictions = model.predict(input_data)

# Obtener el índice de la clase predicha
predicted_index = np.argmax(predictions, axis=1)[0]
predicted_class = class_names[predicted_index]
confidence = predictions[0][predicted_index]

# Mostrar el resultado de manera clara
print(f"Predicción: {predicted_class} (Probabilidad: {confidence:.4f})")

# Mostrar todas las probabilidades de manera ordenada
print("\nProbabilidades de cada clase:")
for i, prob in enumerate(predictions[0]):
    print(f"{class_names[i]}: {prob:.4f}")

# Mostrar la imagen con el resultado
show_image_with_prediction(image_path, predicted_class, confidence)


FileNotFoundError: [Errno 2] No such file or directory: '/content/hombre_8.jpg'

In [8]:
pip install python-multipart

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install fastapi uvicorn tensorflow pillow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# 3.2 Servidor FastAPI embebido en Jupyter

from fastapi import FastAPI, File, UploadFile
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
import nest_asyncio, uvicorn
import numpy as np
from tensorflow import keras
from PIL import Image
import io

# --- 1) Carga (o recarga) tu modelo ---
model = keras.models.load_model("modelo_resnet_entrenado.keras")
class_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# --- 2) Función de preprocesado ---
def preprocess_bytes(image_bytes):
    img = Image.open(io.BytesIO(image_bytes)).convert("RGB")
    img = img.resize((224, 224))
    arr = keras.preprocessing.image.img_to_array(img)
    arr = np.expand_dims(arr, axis=0) / 255.0
    return arr

# --- 3) Crea la app ---
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:3000"],   # esquema + host + puerto
    allow_credentials=True,                    # ← necesario si algún día usas cookies
    allow_methods=["*"],                       # verbos HTTP permitidos
    allow_headers=["*"],                       # encabezados permitidos
)
@app.post("/predict")
async def predict(file: UploadFile = File(...)):
    img_bytes = await file.read()    
    input_data = preprocess_bytes(img_bytes)
    preds = model.predict(input_data)
    idx = int(np.argmax(preds, axis=1)[0])
    mood = class_names[idx]
    confidence = float(preds[0][idx])
    probs = {class_names[i]: float(preds[0][i]) for i in range(len(class_names))}
    # devuelve JSON
    return JSONResponse({
        "predicted_class": mood,
        "confidence": confidence,
        "probabilities": probs
    })

# --- 4) Aplica nest_asyncio y arranca Uvicorn ---
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [20800]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at C:\Users\JOSE\AppData\Roaming\Python\Python39\site-packages\uvicorn\server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "C:\Users\JOSE\AppData\Roaming\Python\Python39\site-packages\uvicorn\main.py", line 580, in run
    server.run()
  File "C:\Users\JOSE\AppData\Roaming\Python\Python39\site-packages\uvicorn\server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "C:\Users\JOSE\AppData\Roaming\Python\Python39\site-packages\nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "C:\Users\JOSE\AppData\Roaming\Python\Python39\site-packages\nest_asyncio.py", line 92, in run_until_complete
    self._

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
INFO:     127.0.0.1:57177 - "POST /predict HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
INFO:     127.0.0.1:57336 - "POST /predict HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
INFO:     127.0.0.1:57450 - "POST /predict HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
INFO:     127.0.0.1:57454 - "POST /predict HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
INFO:     127.0.0.1:57454 - "POST /predict HTTP/1.1" 200 OK
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
INFO:     127.0.0.1:57454 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [20800]
